In [ ]:
from PyQt5 import uic, QtWidgets, QtCore
from time import sleep
import pandas as pd
from PIL import Image, ImageFont, ImageDraw
from datetime import date


# Iniciando a aplicação
app = QtWidgets.QApplication([])

# Carregando as interfaces
primeira_tela = uic.loadUi('home.ui')
segunda_tela = uic.loadUi('cadastro.ui')
terceira_tela = uic.loadUi('email.ui')

# Retirando a borda da interface
primeira_tela.setWindowFlag(QtCore.Qt.FramelessWindowHint)
primeira_tela.setAttribute(QtCore.Qt.WA_TranslucentBackground)
segunda_tela.setWindowFlag(QtCore.Qt.FramelessWindowHint)
segunda_tela.setAttribute(QtCore.Qt.WA_TranslucentBackground)
terceira_tela.setWindowFlag(QtCore.Qt.FramelessWindowHint)
terceira_tela.setAttribute(QtCore.Qt.WA_TranslucentBackground)


# Mostrando a primeira tela
primeira_tela.show()

# Fechando a pagina Home e mostrando a pagina de cadastro
def home():
    primeira_tela.close()
    segunda_tela.show()

# Variaveis do certificado
def cadastro():
    global evento
    global horario
    global carga_horaria
    global dados
    global nome
    global email
    global dict_from_list
    global data
    global assunto_certification

    evento = str(segunda_tela.lineEdit.text())
    horario = str(segunda_tela.lineEdit_2.text())
    carga_horaria = str(segunda_tela.lineEdit_3.text())

    #lendo o dataframe com as colunas "nome" e "email"
    dados = pd.read_csv('dados.csv')        

    #tranformando os dados das colunas "nome" e "email" em uma lista
    nome = list(dados['Nome'])
    email = list(dados['E-mail'])

    #Convertendo todos os nomes em Letra Maiúscula
    nome = [x.upper() for x in nome]

    #criando um dicionário passando os nomes como "key" e os emails como "value"
    dict_from_list = dict(zip(nome, email))

    #especificando a data do evento manualmente
    data = '21/01/2022'

    #corpo do texto da certificação
    assunto_certification = f"Participou da atividade acadêmica intitulada '{evento}', \n\nrealizada no âmbito digital através do canal do Youtube, intitulado 'Uma Dose De Ciência', \n\nde modo remoto, a partir das {horario} horas do dia {data}, com carga horária total de {carga_horaria}h."

    for nome in dict_from_list.keys():
        coord_assunto = (320, 780)
        certificado = Image.open(r'certificado.jpg')
        caminho_fonte1 = r"C:\Windows\FONTS\BASKVILL.TTF"
        caminho_fonte2 = r"C:\Windows\FONTS\BASKVILL.TTF"
        font = ImageFont.truetype(caminho_fonte1, 75)
        desenho = ImageDraw.Draw(certificado)
        W, H = (2550, 1200)
        w, h = desenho.textsize(nome, font=font)
        desenho.text(((W-w)/2,(H-h)/2), nome, font=font , fill= (192,192,192))
        desenho.text(coord_assunto, assunto_certification , font=ImageFont.truetype(caminho_fonte2, 50) , fill= (192,192,192))
        certificado.save(f'certificados/{nome}.pdf')
        
        for c in range(101):
            sleep(0.0001)
            segunda_tela.progressBar.setValue(c)

    segunda_tela.close()
    terceira_tela.show()    

#Enviando certificado para o email
def email():
    import smtplib
    import email.message
    import mimetypes
    import email.mime.application
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    import email
        
    dados = {'Vinícius Carvalho Rosa':'v.rosa@aluno.ifsp.edu.br'}
        
    for nome,email_ in dados.items():
        terceira_tela.label.setText("")
        corpo_email = f"""
        Olá {nome},
        Espero que você esteja bem!
        Segue em anexo a sua declaração de participação da atividade "{evento}" realizada no dia {data}.
        Um grande abraço,
        Vinícius Carvalho Rosa.
        """
        
        msg = email.mime.multipart.MIMEMultipart()

        msg['Subject'] = f"Declaração de Participação - {evento}"
        msg['From'] = 'digite_seu_email@'
        msg['To'] = f'{email_}'

        password = 'digite_sua_senha'

        body = email.mime.text.MIMEText(corpo_email)
        msg.attach(body)
        
        filename = f'certificados/VINICIUS CARVALHO ROSA.pdf'
        fp=open(filename,'rb')
        att = email.mime.application.MIMEApplication(fp.read(),_subtype="pdf")
        fp.close() 
        att.add_header('Content-Disposition','attachment',filename=filename)
        msg.attach(att)


        s = smtplib.SMTP('smtp.gmail.com: 587')
        s.starttls()

        # Login Credentials for sending the mail
        s.login(msg['From'], password)
        s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))    
        
        for c in range(101):
            sleep(0.02)
            terceira_tela.progressBar_2.setValue(c)
        terceira_tela.label.setText('Certificado enviado com sucesso!')
       
    
# Chamando as funções
primeira_tela.pushButton_2.clicked.connect(home)
segunda_tela.pushButton.clicked.connect(cadastro)
terceira_tela.pushButton_2.clicked.connect(email)

# Executando a aplicação
app.exec()
